In [1]:
%pylab inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression

Populating the interactive namespace from numpy and matplotlib


In [ ]:
#take a linear regression by state
